<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

 ## Data sampling with a sliding window with number data
 ## 使用滑动窗口对数字数据进行采样

In [3]:
# 导入版本检查模块
from importlib.metadata import version
# 导入PyTorch库
import torch

# 打印PyTorch版本号
print("torch version:", version("torch"))

torch version: 2.5.0


 To understand the dataloader, which using a sliding window approach, more intuitive, we can consider a dataset that consists of digits only:
 为了更直观地理解使用滑动窗口方法的数据加载器，我们可以考虑一个仅由数字组成的数据集：

```
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 ... 1000
```

In [4]:
# 打开文件用于写入，使用utf-8编码
with open("number-data.txt", "w", encoding="utf-8") as f:
    # 循环生成0-1000的数字
    for number in range(1001):
        # 将每个数字写入文件，数字之间用空格分隔
        f.write(f"{number} ")

 Next, we make a small modification to the `token_ids`: instead of using a tokenizer, we parse the integers directly from the text file:
 接下来，我们对`token_ids`做一个小修改：不使用分词器，而是直接从文本文件中解析整数：

In [5]:
# 从torch.utils.data导入Dataset和DataLoader类
from torch.utils.data import Dataset, DataLoader


# 定义GPTDatasetV1类，继承自Dataset
class GPTDatasetV1(Dataset):
    # 初始化函数,接收文本、分词器、最大长度和步长作为参数
    def __init__(self, txt, tokenizer, max_length, stride):
        # 初始化输入ID列表
        self.input_ids = []
        # 初始化目标ID列表
        self.target_ids = []

        # 修改:不使用分词器,直接将文本分割成整数列表
        # token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        token_ids = [int(i) for i in txt.strip().split()]

        # 使用滑动窗口将文本分成重叠的序列,每个序列长度为max_length
        for i in range(0, len(token_ids) - max_length, stride):
            # 获取输入序列片段
            input_chunk = token_ids[i:i + max_length]
            # 获取目标序列片段(比输入序列向后移动一位)
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # 将输入序列转换为tensor并添加到input_ids列表
            self.input_ids.append(torch.tensor(input_chunk))
            # 将目标序列转换为tensor并添加到target_ids列表
            self.target_ids.append(torch.tensor(target_chunk))

    # 返回数据集长度
    def __len__(self):
        return len(self.input_ids)

    # 根据索引返回数据样本
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [6]:
# 创建数据加载器的函数,接收以下参数:
# txt: 输入文本
# batch_size: 批次大小,默认为4
# max_length: 序列最大长度,默认为256
# stride: 滑动窗口步长,默认为128
# shuffle: 是否打乱数据,默认为True
# drop_last: 是否丢弃最后不完整的批次,默认为True
# num_workers: 数据加载的工作进程数,默认为0
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):

    # 初始化分词器
    # 注释掉原来的tiktoken分词器
    # tokenizer = tiktoken.get_encoding("gpt2")
    # 将分词器设为None,因为我们直接从文本解析整数
    tokenizer = None

    # 使用输入参数创建GPTDatasetV1数据集实例
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建并返回DataLoader实例,设置:
    # - dataset: 上面创建的数据集
    # - batch_size: 批次大小
    # - shuffle: 是否打乱数据
    # - drop_last: 是否丢弃最后不完整的批次
    # - num_workers: 数据加载的工作进程数
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    # 返回创建的数据加载器
    return dataloader

 Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:
 让我们用批次大小为1、上下文长度为4的设置来测试数据加载器:

In [7]:
# 打开并读取number-data.txt文件
# 使用utf-8编码打开文件
# 将文件内容读取到raw_text变量中
with open("number-data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [8]:
# 创建数据加载器,设置:
# - batch_size=1: 每批次1个样本
# - max_length=4: 序列最大长度为4
# - stride=1: 滑动窗口步长为1
# - shuffle=False: 不打乱数据顺序
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

# 创建数据迭代器
data_iter = iter(dataloader)
# 获取第一批数据
first_batch = next(data_iter)
# 打印第一批数据
print(first_batch)

[tensor([[0, 1, 2, 3]]), tensor([[1, 2, 3, 4]])]


In [9]:
# 获取第二批数据
second_batch = next(data_iter)
# 打印第二批数据
print(second_batch)

[tensor([[1, 2, 3, 4]]), tensor([[2, 3, 4, 5]])]


In [10]:
# 获取第三批数据
third_batch = next(data_iter)
# 打印第三批数据
print(third_batch)

[tensor([[2, 3, 4, 5]]), tensor([[3, 4, 5, 6]])]


In [11]:
# 遍历数据加载器中的所有批次
for batch in dataloader:
    pass

# 获取最后一个批次
last_batch = batch
# 打印最后一个批次
print(last_batch)

[tensor([[996, 997, 998, 999]]), tensor([[ 997,  998,  999, 1000]])]


 Now, let's look at the batched inputs:
 现在,让我们看看批量输入:

In [12]:
# 创建数据加载器:
# - batch_size=2: 每批次2个样本
# - max_length=4: 序列最大长度为4 
# - stride=4: 滑动窗口步长为4
# - shuffle=False: 不打乱数据顺序
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)

# 遍历数据加载器中的所有批次
for inputs, targets in dataloader:
    pass

# 打印最后一批次的输入和目标
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[992, 993, 994, 995],
        [996, 997, 998, 999]])

Targets:
 tensor([[ 993,  994,  995,  996],
        [ 997,  998,  999, 1000]])


 Finally, a data loader with shuffling:
 最后,让我们看看带有打乱功能的数据加载器:

In [13]:
# 设置随机种子为123,保证结果可复现
torch.manual_seed(123)
# 创建数据加载器:
# - batch_size=2: 每批次2个样本
# - max_length=4: 序列最大长度为4
# - stride=4: 滑动窗口步长为4 
# - shuffle=True: 打乱数据顺序
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=True)

# 遍历数据加载器中的所有批次
for inputs, targets in dataloader:
    pass

# 打印最后一批次的输入和目标
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[880, 881, 882, 883],
        [112, 113, 114, 115]])

Targets:
 tensor([[881, 882, 883, 884],
        [113, 114, 115, 116]])
